# Семинар: Градиентный спуск. Задачи

In [10]:
from typing import Iterable, List

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Часть 1. Градиентный спуск (вспомним формулы)

Функционал ошибки, который мы применяем в задаче регрессии — Mean Squared Error:

$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2
$$

где $x_i$ — это $i$-ый объект датасета, $y_i$ — правильный ответ для $i$-го объекта, а $w$ — веса нашей линейной модели.

Можно показать, что для линейной модели, функционал ошибки можно записать в матричном виде следующим образом:
$$
Q(w, X, y) =\frac{1}{l} (y - Xw)^T(y-Xw)
$$
или
$$
Q(w, X, y) = \frac{1}{l} || Xw - y ||^2
$$

где $X$ — это матрица объекты-признаки, а $y$ — вектор правильных ответов

Для того чтобы воспользоваться методом градиентного спуска, нам нужно посчитать градиент нашего функционала. Для MSE он будет выглядеть так:

$$
\nabla_w Q(w, X, y) = \frac{2}{l} X^T(Xw-y)
$$

Формула для одной итерации градиентного спуска выглядит следующим образом:

$$
w^t = w^{t-1} - \eta \nabla_{w} Q(w^{t-1}, X, y)
$$

Где $w^t$ — значение вектора весов на $t$-ой итерации, а $\eta$ — параметр learning rate, отвечающий за размер шага.

## Часть 2. Линейная регрессия (продолжение части 1 - используются решения из части 1). 



Создадим класс для линейной регрессии. Он будет использовать интерфейс, знакомый нам из библиотеки `sklearn`.

В методе `fit` мы будем подбирать веса `w` при помощи градиентного спуска нашим методом `gradient_descent`.

В методе `predict` мы будем применять нашу регрессию к датасету, 

**Задание 2.1:** Допишите код в методах `fit` и `predict` класса `LinearRegression_1`

В методе `fit` вам нужно инициализировать веса `w` (например, из члучайного распределения), применить наш `gradient_descent` и сохранить последние веса `w` из траектории.

В методе `predict` вам нужно применить линейную регрессию и вернуть вектор ответов.

Обратите внимание, что объект лосса (функционала ошибки) передаётся в момент инициализации и хранится в `self.loss`. Его нужно использовать в `fit` для `gradient_descent` (например, с `n_iterations: int = 1000`).

In [11]:
import abc


class BaseLoss(abc.ABC):
    """Базовый класс лосса"""

    @abc.abstractmethod
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        raise NotImplementedError

    @abc.abstractmethod
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        raise NotImplementedError

In [12]:
class LinearRegression_1:
    def __init__(self, loss: BaseLoss, lr: float = 0.01) -> None: 
        #loss - функционал ошибки
        #lr - градиентный шаг
        self.loss = loss
        self.lr = lr
        self.w = None  # Веса модели

    def fit(self, X: np.ndarray, y: np.ndarray, n_iterations: int = 1000) -> "LinearRegression_1":
        X = np.asarray(X)
        y = np.asarray(y)
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        # -- YOUR CODE HERE --  
        # Инициализация весов из равномерного распределения
        self.w = np.random.rand(X.shape[1])

        for _ in range(n_iterations):
            # Вычисление градиента
            gradient = (2 / X.shape[0]) * X.T @ (X @ self.w - y)
            # Обновление веса по формуле градиентного спуска
            self.w = self.w - self.lr * gradient
        return self   

    
    def predict(self, X: np.ndarray) -> np.ndarray:
        # Проверяем, что регрессия обучена, то есть, что был вызван fit и в нём был установлен атрибут self.w
        assert hasattr(self, "w"), "Linear regression must be fitted first"
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        # -- YOUR CODE HERE --
        
        # Предсказывание значения
        predictions = X @ self.w
        return predictions

Класс линейной регрессии создан. Более того, мы можем управлять тем, какой функционал ошибки мы оптимизируем, просто передавая разные классы в параметр `loss` при инициализации. 



Будем применять нашу регрессию на реальном датасете. Загрузим датасет с машинами (см. семинар 5_sem-sklearn-knn.ipynb):

In [17]:
import pandas as pd

X_raw = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    header=None,      #в исх. таблице нет названий колонок
    na_values=["?"],  #если ?, то NaN
)

X_raw.head()
X_raw = X_raw[~X_raw[25].isna()].reset_index(drop=True)

In [18]:
y = X_raw[25]
X_raw = X_raw.drop(25, axis=1)

**Задание 2.2:** Обработайте датасет нужными методами, чтобы на нём можно было обучать линейную регрессию (см. семинар 5_sem-sklearn-knn.ipynb):

* Заполните пропуски средними (библиотека SimpleImputer)
* Переведите категориальные признаки в числовые (в методе get_dummies использовать drop_first=True.)
* Разделите датасет на обучающую и тестовую выборку (задать: доля тестовой выборки равна 0.3, `random_state=42`, `shuffle=True`)
* Нормализуйте числовые признаки (при помощи бибилиотеки StandardScaler)

In [19]:
# -- YOUR CODE HERE --
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Разделение на числовые и категориальные признаки
numeric_features = X_raw.select_dtypes(include=[np.number]).columns
categorical_features = X_raw.select_dtypes(include=[object]).columns

# Создание препроцессора для числовых и категориальных признаков
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(drop="first")),
    ]
)

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Разделение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.3, random_state=42, shuffle=True
)

# Применение препроцессора к обучающей выборке
X_train_processed = preprocessor.fit_transform(X_train)

# Применение препроцессора к тестовой выборке
X_test_processed = preprocessor.transform(X_test)

**Задание 2.3:** Обучите написанную вами линейную регрессию на обучающей выборке

In [20]:
# Реализуем класс MSELoss
class MSELoss(BaseLoss):
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        # -- YOUR CODE HERE --
        # Вычислите значение функции потерь при помощи X, y и w и верните его
        l = X.shape[0]
        return np.dot((y - X@w).T, (y - X@w)) / l
  

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        # -- YOUR CODE HERE --
        # Вычислите значение вектора градиента при помощи X, y и w и верните его
        l = X.shape[0]
        return 2 * X.T @ (X @ w - y) / l

Создаем объект линейной регрессии для `MSELoss`:

In [21]:
lr_1 = LinearRegression_1(MSELoss()) #создаем регрессор


In [23]:
# -- YOUR CODE HERE --

# Обучаем регрессор на обучающей выборке
lr_1.fit(X_train_processed, y_train)

# Предсказываем значения на обучающей выборке
y_train_pred = lr_1.predict(X_train_processed)

**Задание 2.4:** Посчитайте ошибку обученной регрессии на обучающей и тестовой выборке при помощи методов `mean_squared_error` и `r2_score` из `sklearn.metrics`.

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [25]:
# -- YOUR CODE HERE --
from sklearn.metrics import mean_squared_error, r2_score

# Создаем объект линейной регрессии
lr_1 = LinearRegression_1(MSELoss())

# Обучаем регрессор на обучающей выборке
lr_1.fit(X_train_processed, y_train)

# Предсказываем значения на обучающей выборке
y_train_pred = lr_1.predict(X_train_processed)

# Предсказываем значения на тестовой выборке
y_test_pred = lr_1.predict(X_test_processed)

# Вычисляем MSE на обучающей выборке
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"MSE на обучающей выборке: {mse_train}")

# Вычисляем R^2 на обучающей выборке
r2_train = r2_score(y_train, y_train_pred)
print(f"R^2 на обучающей выборке: {r2_train}")

# Вычисляем MSE на тестовой выборке
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"MSE на тестовой выборке: {mse_test}")

# Вычисляем R^2 на тестовой выборке
r2_test = r2_score(y_test, y_test_pred)
print(f"R^2 на тестовой выборке: {r2_test}")

MSE на обучающей выборке: 4027842.3495557
R^2 на обучающей выборке: 0.916401216290347
MSE на тестовой выборке: 11062903.552126575
R^2 на тестовой выборке: 0.8832924356609586


Добавим к модели L2 регуляризацию (для борьбы с переобучением). Для этого нам нужно создать новый класс для нового функционала ошибки и его градиента.

Формула функционала ошибки для MSE с L2 регуляризацией выглядит так:
$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2 + \lambda ||w||^2
$$

Или в матричном виде:

$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2 + \lambda ||w||^2,
$$

где $\lambda$ — коэффициент регуляризации

Заметим, что (удобно для вычислений):
$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2+\lambda ||w||^2 =\frac{1}{l} (y - Xw)^T(y-Xw)+ \lambda w^Tw.
$$

Градиент $\nabla_w Q(w, X, y)$ выглядит так:

$$
\nabla_w Q(w, X, y) = \frac{2}{\ell} X^T(Xw-y) + 2 \lambda w
$$

**Задание 2.5:** Реализуйте класс `MSEL2Loss`

Он должен вычислять значение функционала ошибки (лосс) $
Q(w, X, y)$ и его градиент $\nabla_w Q(w, X, y)$ по формулам (выше).

Подсказка: обратите внимание, что последний элемент вектора `w` — это bias (сдвиг) (в классе `LinearRegression` к матрице `X` добавляется колонка из единиц — константный признак). bias регуляризовать не нужно. Поэтому не забудьте убрать последний элемент из `w` при подсчёте слагаемого $\lambda||w||^2$ в `calc_loss` и занулить его при подсчёте слагаемого $2 \lambda w$ в `calc_grad`

In [27]:
class MSEL2Loss(BaseLoss):
    def __init__(self, coef: float = 1.0):
        """
        :param coef: коэффициент регуляризации (лямбда в формуле)
        """
        self.coef = coef

    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета. Последний признак константный.
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии. Последний вес -- bias.
        :output: число -- значения функции потерь
        """    
        # -- YOUR CODE HERE --
        # Вычислите значение функции потерь при помощи X, y и w и верните его
        
        predictions = np.dot(X, w)
        residuals = predictions - y
        loss = np.mean(residuals**2) + self.coef * np.sum(w[:-1]**2)  # Учитываем регуляризацию, не затрагивая bias
        return loss

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :output: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        # -- YOUR CODE HERE --
        # Вычислите значение вектора градиента при помощи X, y и w и верните его
        
        predictions = np.dot(X, w)
        residuals = predictions - y
        gradient = (2 / len(y)) * np.dot(X.T, residuals) + 2 * self.coef * np.hstack([w[:-1], [0]])  # Зануляем градиент для bias
        return gradient

In [28]:
#Проверка:
#Создадим объект лосса
losst = MSEL2Loss()

# Создадим датасет
Xt = np.arange(200).reshape(20, 10)
yt = np.arange(20)

# Создадим вектор весов
wt = np.arange(10)

#print(Xt)
#print(yt)
#print(wt)

# Выведем значение лосса и градиента на этом датасете с этим вектором весов
print(losst.calc_loss(Xt, yt, wt))
print(losst.calc_grad(Xt, yt, wt))

# Проверка, что методы реализованы правильно
assert losst.calc_loss(Xt, yt, wt) == 27410487.5, "Метод calc_loss реализован неверно" 
assert np.allclose(
    losst.calc_grad(Xt, yt, wt),
    np.array(
        [
            1163180.0,
            1172283.0,
            1181386.0,
            1190489.0,
            1199592.0,
            1208695.0,
            1217798.0,
            1226901.0,
            1236004.0,
            1245089.0,
        ]
    ),
), "Метод calc_grad реализован неверно"

print("Всё верно!")

27410487.5
[1163180. 1172283. 1181386. 1190489. 1199592. 1208695. 1217798. 1226901.
 1236004. 1245089.]
Всё верно!


Теперь мы можем использовать лосс с l2 регуляризацией в нашей регрессии. Пусть:

In [29]:
lr_1_l2 = LinearRegression_1(MSEL2Loss(0.1))

**Задание 2.6:** Обучите регрессию с лоссом `MSEL2Loss`. Попробуйте использовать другие коэффициенты регуляризации. Получилось ли улучшить разультат на тестовой выборке? Сравните результат применения регрессии с регуляризацией к обучающей и тестовой выборкам с результатом применения регрессии без регуляризации к тем же выборкам.(Для оценки качества использовать методы `mean_squared_error` и `r2_score` из `sklearn.metrics`).

In [40]:
# -- YOUR CODE HERE --

# Создаем объект линейной регрессии с L2-регуляризацией, коэффициент регуляризации 1000
lr_1_l2 = LinearRegression_1(MSEL2Loss(coef=1000))

# Обучаем регрессор на обучающей выборке
lr_1_l2.fit(X_train_processed, y_train)

# Предсказываем значения на тестовой выборке
y_pred_l2 = lr_1_l2.predict(X_test_processed)

# Сравниваем результаты с регрессией без регуляризации
mse_test_l2 = mean_squared_error(y_test, y_pred_l2)
r2_test_l2 = r2_score(y_test, y_pred_l2)

print(f"MSE на тестовой выборке с L2-регуляризацией: {mse_test_l2}")
print(f"R^2 на тестовой выборке с L2-регуляризацией: {r2_test_l2}")

MSE на тестовой выборке с L2-регуляризацией: 11063024.216822745
R^2 на тестовой выборке с L2-регуляризацией: 0.8832911627146001


In [39]:
# Коэффициент регуляризации 0.001
lr_1_l2 = LinearRegression_1(MSEL2Loss(coef=0.001))

lr_1_l2.fit(X_train_processed, y_train)

y_pred_l2 = lr_1_l2.predict(X_test_processed)

mse_test_l2 = mean_squared_error(y_test, y_pred_l2)
r2_test_l2 = r2_score(y_test, y_pred_l2)

print(f"MSE на тестовой выборке с L2-регуляризацией: {mse_test_l2}")
print(f"R^2 на тестовой выборке с L2-регуляризацией: {r2_test_l2}")

MSE на тестовой выборке с L2-регуляризацией: 11062595.026644185
R^2 на тестовой выборке с L2-регуляризацией: 0.8832956904355679


Как можно заменить, значение коэффициента регуляризации практически не влияет на модель.